### Importing Libraries

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import sys
import collections
sys.path.extend(['../input/bert-joint-baseline/'])

import bert_utils
from bert_utils import AnswerType
import modeling 

import tokenization
import json

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


### Splitting the Dataset

In [2]:
on_kaggle_server = os.path.exists('/kaggle')
nq_test_file = '../input/tensorflow2-question-answering/simplified-nq-test.jsonl' 
nq_train_file = '../input/tensorflow2-question-answering/simplified-nq-train.jsonl'
public_dataset = os.path.getsize(nq_test_file)<20_000_000
private_dataset = os.path.getsize(nq_test_file)>=20_000_000

In [3]:
if True:
    import importlib
    importlib.reload(bert_utils)

In [4]:
config = {'attention_probs_dropout_prob':0.1,
'hidden_act':'gelu', # 'gelu',
'hidden_dropout_prob':0.1,
'hidden_size':1024,
'initializer_range':0.02,
'intermediate_size':4096,
'max_position_embeddings':512,
'num_attention_heads':16,
'num_hidden_layers':24,
'type_vocab_size':2,
'vocab_size':30522}

In [5]:
class TDense(tf.keras.layers.Layer):
    def __init__(self,
                 output_size,
                 kernel_initializer=None,
                 bias_initializer="zeros",
                **kwargs):
        super().__init__(**kwargs)
        self.output_size = output_size
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
    def build(self,input_shape):
        dtype = tf.as_dtype(self.dtype or tf.keras.backend.floatx())
        if not (dtype.is_floating or dtype.is_complex):
          raise TypeError("Unable to build `TDense` layer with "
                          "non-floating point (and non-complex) "
                          "dtype %s" % (dtype,))
        input_shape = tf.TensorShape(input_shape)
        if tf.compat.dimension_value(input_shape[-1]) is None:
          raise ValueError("The last dimension of the inputs to "
                           "`TDense` should be defined. "
                           "Found `None`.")
        last_dim = tf.compat.dimension_value(input_shape[-1])
        self.input_spec = tf.keras.layers.InputSpec(min_ndim=3, axes={-1: last_dim})
        self.kernel = self.add_weight(
            "kernel",
            shape=[self.output_size,last_dim],
            initializer=self.kernel_initializer,
            dtype=self.dtype,
            trainable=True)
        self.bias = self.add_weight(
            "bias",
            shape=[self.output_size],
            initializer=self.bias_initializer,
            dtype=self.dtype,
            trainable=True)
        super(TDense, self).build(input_shape)
    def call(self,x):
        return tf.matmul(x,self.kernel,transpose_b=True)+self.bias
    
def mk_model(config):
    seq_len = config['max_position_embeddings']
    unique_id  = tf.keras.Input(shape=(1,),dtype=tf.int64,name='unique_id')
    input_ids   = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='input_ids')
    input_mask  = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='input_mask')
    segment_ids = tf.keras.Input(shape=(seq_len,),dtype=tf.int32,name='segment_ids')
    BERT = modeling.BertModel(config=config,name='bert')
    pooled_output, sequence_output = BERT(input_word_ids=input_ids,
                                          input_mask=input_mask,
                                          input_type_ids=segment_ids)
    
    logits = TDense(2,name='logits')(sequence_output)
    start_logits,end_logits = tf.split(logits,axis=-1,num_or_size_splits= 2,name='split')
    start_logits = tf.squeeze(start_logits,axis=-1,name='start_squeeze')
    end_logits   = tf.squeeze(end_logits,  axis=-1,name='end_squeeze')
    
    ans_type      = TDense(5,name='ans_type')(pooled_output)
    return tf.keras.Model([input_ for input_ in [unique_id,input_ids,input_mask,segment_ids] 
                           if input_ is not None],
                          [unique_id,start_logits,end_logits,ans_type],
                          name='bert-baseline')    

In [6]:
model= mk_model(config)

In [7]:
model.summary()

Model: "bert-baseline"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (BertModel)                ((None, 1024), (None 335141888   input_ids[0][0]                  
                                                                 input_mask[0][0]     

### Checkpoint

In [8]:
cpkt = tf.train.Checkpoint(model=model)
cpkt.restore('../input/bert-joint-baseline/model_cpkt-1').assert_consumed()

### Setting the Flags

In [6]:
class DummyObject:
    def __init__(self,**kwargs):
        self.__dict__.update(kwargs)

FLAGS=DummyObject(skip_nested_contexts=True, #True
                  max_position=50,
                  max_contexts=48,
                  max_query_length=64,
                  max_seq_length=512, #512
                  doc_stride=128,
                  include_unknowns=0.02, #0.02
                  n_best_size=5, #20
                  max_answer_length=30, #30
                  
                  warmup_proportion=0.1,
                  learning_rate=1e-5,
                  num_train_epochs=3.0,
                  train_batch_size=32,
                  num_train_steps=100000,
                  num_warmup_steps=10000,
                  max_eval_steps=100,
                  use_tpu=False,
                  eval_batch_size=8, 
                  max_predictions_per_seq=20)

In [7]:
import tqdm
eval_records = "../input/bert-joint-baseline/nq-test.tfrecords"

if on_kaggle_server and private_dataset:
    eval_records='nq-test.tfrecords'
if not os.path.exists(eval_records):
    
    eval_writer = bert_utils.FeatureWriter(
        filename=os.path.join(eval_records),
        is_training=False)

    tokenizer = tokenization.FullTokenizer(vocab_file='../input/bert-joint-baseline/vocab-nq.txt', 
                                           do_lower_case=True)

    features = []
    convert = bert_utils.ConvertExamples2Features(tokenizer=tokenizer,
                                                   is_training=False,
                                                   output_fn=eval_writer.process_feature,
                                                   collect_stat=False)

    n_examples = 0
    tqdm_notebook= tqdm.tqdm_notebook if not on_kaggle_server else None
    for examples in bert_utils.nq_examples_iter(input_file=nq_test_file, 
                                           is_training=False,
                                           tqdm=tqdm_notebook):
        for example in examples:
            n_examples += convert(example)

    eval_writer.close()
    print('number of test examples: %d, written to file: %d' % (n_examples,eval_writer.num_features))

In [8]:
seq_length = FLAGS.max_seq_length #config['max_position_embeddings']
name_to_features = {
      "unique_id": tf.io.FixedLenFeature([], tf.int64),
      "input_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.io.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.io.FixedLenFeature([seq_length], tf.int64),
  }

def _decode_record(record, name_to_features=name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.io.parse_single_example(serialized=record, features=name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if name != 'unique_id': #t.dtype == tf.int64:
            t = tf.cast(t, dtype=tf.int64)
        example[name] = t

    return example

def _decode_tokens(record):
    return tf.io.parse_single_example(serialized=record, 
                                      features={
                                          "unique_id": tf.io.FixedLenFeature([], tf.int64),
                                          "token_map" :  tf.io.FixedLenFeature([seq_length], tf.int64)
                                      })
      


In [9]:
raw_ds = tf.data.TFRecordDataset(eval_records)
token_map_ds = raw_ds.map(_decode_tokens)
decoded_ds = raw_ds.map(_decode_record)
ds = decoded_ds.batch(batch_size=32,drop_remainder=False) 

In [11]:
result=model.predict_generator(ds,verbose=1)

NameError: name 'model' is not defined

In [12]:
np.savez_compressed('bert-joint-baseline-output.npz',
                    **dict(zip(['uniqe_id','start_logits','end_logits','answer_type_logits'],
                               result)))

NameError: name 'result' is not defined

In [13]:
# class ScoreSummary(object):
#     def __init__(self):
#         self.predicted_label = None
#         self.short_span_score = None
#         self.cls_token_score = None
#         self.answer_type_logits = None

In [10]:
ScoreSummary = collections.namedtuple("ScoreSummary", ["short_span_score", "cls_token_score",
                                                       "answer_type_logits", "answer_type",
                                                       "start_logits", "end_logits", "unique_id",
                                                      "start_idx_in_chunk", "end_idx_in_chunk"])
def empty_score_summary():
    return ScoreSummary(None, None, None, None, None, None, None, None, None)
    
Span = collections.namedtuple("Span", ["start_token_idx", "end_token_idx", "score", "summary"])

In [11]:
class EvalExample(object):
  """Eval data available for a single example."""
  def __init__(self, example_id, candidates):
    self.example_id = example_id
    self.candidates = candidates
    self.results = {}
    self.features = {}

In [12]:
def get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(
      enumerate(logits[1:], 1), key=lambda x: x[1], reverse=True)
  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes

def top_k_indices(logits,n_best_size,token_map):
    indices = np.argsort(logits[1:])+1
    indices = indices[token_map[indices]!=-1]
    return indices[-n_best_size:]

In [13]:
def remove_duplicates(span):
    start_end = []
    for s in span:
        cont = 0
        if not start_end:
            start_end.append(s)
            cont += 1
        else:
            for i in range(len(start_end)):
                if start_end[i].start_token_idx == s.start_token_idx and \
                start_end[i].end_token_idx == s.end_token_idx:
                    cont += 1
        if cont == 0:
            start_end.append(s)
            
    return start_end

In [14]:
def get_short_long_span_answers(predictions, example):
    
    sorted_predictions = sorted(predictions, reverse=True)
    short_span = []
    long_span = []
    for prediction in sorted_predictions:
        score, _, summary, start_span, end_span = prediction
        # get scores > zero
        if score > 0:
            short_span.append(Span(int(start_span), int(end_span), float(score), summary))

    short_span = remove_duplicates(short_span)

    for s in range(len(short_span)):
        for c in example.candidates:
            start = short_span[s].start_token_idx
            end = short_span[s].end_token_idx
            ## print(c['top_level'],c['start_token'],start,c['end_token'],end)
            if c["top_level"] and c["start_token"] <= start and c["end_token"] >= end:
                long_span.append(Span(int(c["start_token"]), int(c["end_token"]),
                                      float(short_span[s].score), short_span[s].summary))
                break
    long_span = remove_duplicates(long_span)
    
    if not long_span:
        long_span = [Span(-1, -1, -10000.0, empty_score_summary())]
    if not short_span:
        short_span = [Span(-1, -1, -10000.0, empty_score_summary())]
        
    
    return short_span, long_span

In [15]:
def compute_predictions(example):
    """Converts an example into an NQEval object for evaluation."""
    predictions = []
    n_best_size = FLAGS.n_best_size
    max_answer_length = FLAGS.max_answer_length
    i = 0
    for unique_id, result in example.results.items():
        if unique_id not in example.features:
            raise ValueError("No feature found with unique_id:", unique_id)
        token_map = np.array(example.features[unique_id]["token_map"]) #.int64_list.value
        start_indexes = top_k_indices(result.start_logits,n_best_size,token_map)
        if len(start_indexes)==0:
            continue
        end_indexes   = top_k_indices(result.end_logits,n_best_size,token_map)
        if len(end_indexes)==0:
            continue
        indexes = np.array(list(np.broadcast(start_indexes[None],end_indexes[:,None])))  
        indexes = indexes[(indexes[:,0] < indexes[:,1]) * (indexes[:,1] - indexes[:,0] < max_answer_length)]
        for _, (start_index,end_index) in enumerate(indexes):  
            short_span_score = (
                result.start_logits[start_index] +
                result.end_logits[end_index])
            cls_token_score = (
                result.start_logits[0] + result.end_logits[0])
            answer_type_logits = result.answer_type_logits - result.answer_type_logits.mean()
            answer_type = int(np.argmax(answer_type_logits))
            
            start_logits = list(result.start_logits)
            end_logits = list(result.end_logits)
            start_idx_in_chunk = start_index
            end_idx_in_chunk = end_index
            
            summary = ScoreSummary(short_span_score, cls_token_score, list(answer_type_logits),
                                   answer_type, start_logits, end_logits, unique_id,
                                   start_idx_in_chunk, end_idx_in_chunk)
            
            start_span = token_map[start_index]
            end_span = token_map[end_index] + 1

            # Span logits minus the cls logits seems to be close to the best.
            score = summary.short_span_score - summary.cls_token_score
            predictions.append((score, i, summary, start_span, end_span))
            i += 1 # to break ties

    # Default empty prediction.
    short_spans = [Span(-1, -1, -10000.0, empty_score_summary())]
    long_spans  = [Span(-1, -1, -10000.0, empty_score_summary())]

    if predictions:
        short_spans, long_spans = get_short_long_span_answers(predictions, example)
      
    predictions_summary = {
        "example_id": int(example.example_id),
        "short_spans": short_spans,
        "long_spans": long_spans
    }

    return predictions_summary

In [16]:
def compute_pred_dict(candidates_dict, dev_features, raw_results,tqdm=None):
    """Computes official answer key from raw logits."""
    raw_results_by_id = [(int(res.unique_id),1, res) for res in raw_results]

    examples_by_id = [(int(k),0,v) for k, v in candidates_dict.items()]
  
    features_by_id = [(int(d['unique_id']),2,d) for d in dev_features] 
  
    # Join examples with features and raw results.
    examples = []
    print('merging examples...')
    merged = sorted(examples_by_id + raw_results_by_id + features_by_id)
    print('done.')
    for idx, type_, datum in merged:
        if type_==0: #isinstance(datum, list):
            examples.append(EvalExample(idx, datum))
        elif type_==2: #"token_map" in datum:
            examples[-1].features[idx] = datum
        else:
            examples[-1].results[idx] = datum

    # Construct prediction objects.
    print('Computing predictions...')
   
    nq_pred_dict = {}
    if tqdm is not None:
        examples = tqdm(examples)
    for e in examples:
        predictions_summary = compute_predictions(e)
        nq_pred_dict[e.example_id] = predictions_summary
    return nq_pred_dict

In [17]:
def read_candidates_from_one_split(input_path):
  """Read candidates from a single jsonl file."""
  candidates_dict = {}
  print("Reading examples from: %s" % input_path)
  if input_path.endswith(".gz"):
    with gzip.GzipFile(fileobj=tf.io.gfile.GFile(input_path, "rb")) as input_file:
      for index, line in enumerate(input_file):
        e = json.loads(line)
        candidates_dict[e["example_id"]] = e["long_answer_candidates"]
        
  else:
    with tf.io.gfile.GFile(input_path, "r") as input_file:
      for index, line in enumerate(input_file):
        e = json.loads(line)
        candidates_dict[e["example_id"]] = e["long_answer_candidates"] # testar juntando com question_text
  return candidates_dict

In [18]:
def read_candidates(input_pattern):
  """Read candidates with real multiple processes."""
  input_paths = tf.io.gfile.glob(input_pattern)
  final_dict = {}
  for input_path in input_paths:
    final_dict.update(read_candidates_from_one_split(input_path))
  return final_dict

In [19]:
# Loading eval_features
import pickle
eval_features = []
with open('../output/1k/eval_features.pkl', 'rb') as f:
    eval_features = pickle.load(f)
len(eval_features)

28308

In [20]:
# Loading result
# 'uniqe_id','start_logits','end_logits','answer_type_logits'
result_file = np.load('../output/1k/bert-joint-baseline-output.npz')
result = [result_file['uniqe_id'], result_file['start_logits'], result_file['end_logits'],
          result_file['answer_type_logits']]

In [21]:
all_results = [bert_utils.RawResult(*x) for x in zip(*result)]
    
print ("Going to candidates file")

# for debug
#candidates_dict = read_candidates('../input/tensorflow2-question-answering/simplified-nq-test.jsonl')
candidates_dict = read_candidates('../output/1k/nq-train-part.jsonl')

print ("setting up eval features")

# for debug
#eval_features = list(token_map_ds)

print ("compute_pred_dict")

#tqdm_notebook= tqdm.tqdm_notebook
tqdm_notebook= None
nq_pred_dict = compute_pred_dict(candidates_dict,
                                 eval_features,
                                 all_results,
                                 tqdm=tqdm_notebook)

predictions_json = {"predictions": list(nq_pred_dict.values())}

# print ("writing json")

# with tf.io.gfile.GFile('../output/predictions-local.json', "w") as f:
#     json.dump(predictions_json, f, indent=4)
print('done!')

Going to candidates file
Reading examples from: ../output/1k/nq-train-part.jsonl
setting up eval features
compute_pred_dict
merging examples...
done.
Computing predictions...
done!


In [22]:
# for debug
#answers_df = pd.read_json("../working/predictions.json")
# answers_df = pd.read_json("../output/predictions-local.json")
answers_df = pd.DataFrame.from_dict(predictions_json)
answers_df.head()

,predictions
0,"{'example_id': -9209839852162522524, 'short_sp..."
1,"{'example_id': -9188885911445781635, 'short_sp..."
2,"{'example_id': -9111510312671706854, 'short_sp..."
3,"{'example_id': -9110190923673509457, 'short_sp..."
4,"{'example_id': -9100123296297706673, 'short_sp..."


In [23]:
##### {long score > 2, cont = 5 | short score > 2, cont = 5} = 0.18
# { long score > 2, cont = 5 | short score > 6, cont = 5}
# { long score > 2, cont = 1 | short score > 6, cont = 5}

def df_long_index_score(df):
    answers = []
    cont = 0
    for e in df['long_spans']:
        # if score > 2
        if e[2] > 3: 
            index = {}
            index['start'] = e[0]
            index['end'] = e[1]
            index['score'] = e[2]
            answers.append(index)
            cont += 1
        # number of answers
        if cont == 1:
            break
            
    return answers

def df_short_index_score(df):
    answers = []
    cont = 0
    for e in df['short_spans']:
        # if score > 2
        if e[2] > 8:
            index = {}
            index['start'] = e[0]
            index['end'] = e[1]
            index['score'] = e[2]
            answers.append(index)
            cont += 1
        # number of answers
        if cont == 1:
            break
            
    return answers

def df_example_id(df):
    return df['example_id']

In [24]:
answers_df['example_id'] = answers_df['predictions'].apply(df_example_id)

answers_df['long_indexes_and_scores'] = answers_df['predictions'].apply(df_long_index_score)

answers_df['short_indexes_and_scores'] = answers_df['predictions'].apply(df_short_index_score)

answers_df.head()

,predictions,example_id,long_indexes_and_scores,short_indexes_and_scores
0,"{'example_id': -9209839852162522524, 'short_sp...",-9209839852162522524,[],[]
1,"{'example_id': -9188885911445781635, 'short_sp...",-9188885911445781635,[],[]
2,"{'example_id': -9111510312671706854, 'short_sp...",-9111510312671706854,"[{'start': 189, 'end': 282, 'score': 8.9754791...","[{'start': 262, 'end': 266, 'score': 8.9754791..."
3,"{'example_id': -9110190923673509457, 'short_sp...",-9110190923673509457,"[{'start': 201, 'end': 329, 'score': 8.1631450...","[{'start': 202, 'end': 215, 'score': 8.1631450..."
4,"{'example_id': -9100123296297706673, 'short_sp...",-9100123296297706673,"[{'start': 519, 'end': 620, 'score': 12.078039...","[{'start': 598, 'end': 601, 'score': 12.078039..."


In [25]:
#answers_df = answers_df.drop(['predictions'], axis=1)
answers_df.head()

,predictions,example_id,long_indexes_and_scores,short_indexes_and_scores
0,"{'example_id': -9209839852162522524, 'short_sp...",-9209839852162522524,[],[]
1,"{'example_id': -9188885911445781635, 'short_sp...",-9188885911445781635,[],[]
2,"{'example_id': -9111510312671706854, 'short_sp...",-9111510312671706854,"[{'start': 189, 'end': 282, 'score': 8.9754791...","[{'start': 262, 'end': 266, 'score': 8.9754791..."
3,"{'example_id': -9110190923673509457, 'short_sp...",-9110190923673509457,"[{'start': 201, 'end': 329, 'score': 8.1631450...","[{'start': 202, 'end': 215, 'score': 8.1631450..."
4,"{'example_id': -9100123296297706673, 'short_sp...",-9100123296297706673,"[{'start': 519, 'end': 620, 'score': 12.078039...","[{'start': 598, 'end': 601, 'score': 12.078039..."


In [26]:
def create_answer(entry):
    answer = []
    for e in entry:
        answer.append(str(e['start']) + ':'+ str(e['end']))
    if not answer:
        answer = ""
    return ", ".join(answer)


In [27]:
answers_df["long_answer"] = answers_df['long_indexes_and_scores'].apply(create_answer)
answers_df["short_answer"] = answers_df['short_indexes_and_scores'].apply(create_answer)
answers_df["example_id"] = answers_df['example_id'].apply(lambda q: str(q))

long_answers = dict(zip(answers_df["example_id"], answers_df["long_answer"]))
short_answers = dict(zip(answers_df["example_id"], answers_df["short_answer"]))

answers_df.head()

,predictions,example_id,long_indexes_and_scores,short_indexes_and_scores,long_answer,short_answer
0,"{'example_id': -9209839852162522524, 'short_sp...",-9209839852162522524,[],[],,
1,"{'example_id': -9188885911445781635, 'short_sp...",-9188885911445781635,[],[],,
2,"{'example_id': -9111510312671706854, 'short_sp...",-9111510312671706854,"[{'start': 189, 'end': 282, 'score': 8.9754791...","[{'start': 262, 'end': 266, 'score': 8.9754791...",189:282,262:266
3,"{'example_id': -9110190923673509457, 'short_sp...",-9110190923673509457,"[{'start': 201, 'end': 329, 'score': 8.1631450...","[{'start': 202, 'end': 215, 'score': 8.1631450...",201:329,202:215
4,"{'example_id': -9100123296297706673, 'short_sp...",-9100123296297706673,"[{'start': 519, 'end': 620, 'score': 12.078039...","[{'start': 598, 'end': 601, 'score': 12.078039...",519:620,598:601


In [28]:
answers_df = answers_df.drop(['long_indexes_and_scores', 'short_indexes_and_scores'], axis=1)
answers_df.head()

,predictions,example_id,long_answer,short_answer
0,"{'example_id': -9209839852162522524, 'short_sp...",-9209839852162522524,,
1,"{'example_id': -9188885911445781635, 'short_sp...",-9188885911445781635,,
2,"{'example_id': -9111510312671706854, 'short_sp...",-9111510312671706854,189:282,262:266
3,"{'example_id': -9110190923673509457, 'short_sp...",-9110190923673509457,201:329,202:215
4,"{'example_id': -9100123296297706673, 'short_sp...",-9100123296297706673,519:620,598:601


In [29]:
answers_df.query("short_answer == 'NO'")

,predictions,example_id,long_answer,short_answer


### Generating the Submission File

In [30]:
# for debug
#answers_df.to_csv('test_answers_df.csv', index=False)
answers_df.to_csv('../output/test_answers_df_v1.csv', index=False)

In [31]:
sample_submission = pd.read_csv("../input/tensorflow2-question-answering/sample_submission.csv")

long_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_long")].apply(lambda q: long_answers[q["example_id"].replace("_long", "")], axis=1)
short_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_short")].apply(lambda q: short_answers[q["example_id"].replace("_short", "")], axis=1)

sample_submission.loc[sample_submission["example_id"].str.contains("_long"), "PredictionString"] = long_prediction_strings
sample_submission.loc[sample_submission["example_id"].str.contains("_short"), "PredictionString"] = short_prediction_strings


KeyError: ('-1011141123527297803', 'occurred at index 0')

In [35]:
sample_submission

,example_id,PredictionString
0,-1011141123527297803_long,
1,-1011141123527297803_short,
2,-1028916936938579349_long,
3,-1028916936938579349_short,
4,-1055197305756217938_long,221:335
...,...,...
687,930196817123445627_short,
688,934950704129184964_long,496:616
689,934950704129184964_short,515:517
690,958723574737344087_long,938:2597
